In [ ]:
import mne
import numpy as np
import random
random.seed(42)

import warnings
warnings.filterwarnings('ignore')

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import pickle

import import_ipynb
from CommonFunctions import loadData
from sklearn.preprocessing import StandardScaler

from mne.decoding import Vectorizer, get_coef

In [ ]:
#File paths
meg_MainFolder = "..\Data\MEG_Data\Data="
figures_MainFolder = "..\Figures\Others\Data="
results_MainFolder = "..\Results\Data="
classifiers_MainFolder = "..\Classifiers\Data="

tmin, tmax = -0.1, 0.6
print('tmin = ', tmin)
print('tmax = ', tmax)

# Decide the time limit based on the time range of the data
tlim = 0
if tmin == -0.8:
    tlim = 140
    peak_indices = [90, 120]
elif tmin == -0.1:
    tlim = 70
    peak_indices = [20, 50]
    
print('time limit: ', tlim)
print('time range of classifiers: ', peak_indices)


dataFolder = meg_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Data folder: ', dataFolder)

figuresFolder = figures_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Figures folder: ', figuresFolder)

results_folder = results_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Results folder: ', results_folder)

clsfFolder = classifiers_MainFolder + str(tmin) + '_' + str(tmax) + '\\'
print('Classifiers folder: ', clsfFolder)

sensors = 'mag'


In [ ]:
filename_ext = ''
if tmin == -0.8 and tmax == 0.6:
    filename_ext = '-elongated'
elif tmin == -0.8 and tmax == 1:
    filename_ext = '-elongated_2'
    
print('filename ext: ', filename_ext)

In [ ]:
s_id_list = ['13', '16', '17', '18', '21', '23', '26', '28', '29', '30', '31', '32', '33', '34',
             '35', '36', '38', '39', '40', '41', '42']

print('Number of subjects: ', len(s_id_list))
task_name = 'all_predLevel' #"all_predLevel" #'all_incorrVScorr'# 'all_conf'
print('Task name: ', task_name)

In [ ]:
epochs_list, bestParams = [], []
real_events = ['living_real_8', 'object_real_8', 'living_real_9', 'object_real_9', 'living_real_10', 'object_real_10']
for s_id in s_id_list:
    if int(s_id) < 23:
        fname = dataFolder + 'S' + s_id + '\\' + s_id + '_2_tsss_mc_trans_'+sensors+ '_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'
    else:
        fname = dataFolder + 'S' + s_id + '\\block_2_tsss_mc_trans_'+sensors+ '_nobase-epochs_afterICA'+filename_ext+'_manually_AR_resampled.fif'

    epochs = loadData(s_id, sensors, fname, resampled=True)
    epochs_real = epochs[real_events]
    epochs_list.append(epochs_real)
    

In [ ]:
print('Shape of data: ', epochs_list[0]._data.shape)

#### Load Classifiers from file

In [ ]:

models_list = []
for s_id in s_id_list:
    filename = clsfFolder+'S'+s_id+'\\'+s_id+'_clsf_'+task_name+'_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_sm.sav'
    print('Classifier is loaded from ', filename)
    loaded_models = [] # store each loaded model in a list in case a model is not available (whch will return empty list)
    with open(filename, "rb") as f:
        while True:
            try:
                loaded_models.append(pickle.load(f))
            except EOFError:
                break
    models_list.append(loaded_models)

In [ ]:
models_list[0:3]

In [ ]:
activations_group = []

In [ ]:
def plotActivations(s_id, model, epochs, timePoint, isClsf=False, standardize=False):
    print("s_id: ", s_id)
    print('Time point: ', epochs.times[timePoint]) 
    print("Meg data shape: ", epochs.get_data()[:,:,timePoint].shape)
        
    # Get classifier weights
    coef = get_coef(model, 'coef_', inverse_transform=True)
    
    # Below part first computes standardizes
    if standardize == True:
        coef_mean = np.mean(coef)
        coef_std = np.std(coef)
        
        # Standardize the weights
        coef = (coef-coef_mean)/coef_std
        print('shape of coef: ', coef.shape)
    
    
    # Plotting activation maps
    if isClsf  == False: # activation maps
        # Multiplying classifier weights with covariance of data to compute activation maps
        
        epochs_tmp = epochs.copy()
        # crop the data at the time point of max performance
        epochs_tmp.crop(tmin=epochs.times[timePoint], tmax=epochs.times[timePoint])
        cov_tmp = mne.compute_covariance(epochs_tmp, verbose=False)
        activations = np.dot(coef, cov_tmp.data)

        del cov_tmp 


    #plotting classifier weights 
    else:
        activations = coef
        
    # Simulate evoked data with activation to plot on topo maps
    evoked = mne.EvokedArray(np.transpose(activations), epochs.info, tmin=0)
    evoked.plot_topomap([0], show=False, time_unit='s', ch_type='mag', colorbar=True,
                        outlines='head', scalings=1e-9)
            

    # Save the plot
    if isClsf ==True:
        plotname = figuresFolder+'S'+s_id+'\\'+s_id+'_clsfWeights_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_'+task_name+'_clsfWeights.png'
    else:
        plotname = figuresFolder+'S'+s_id+'\\'+s_id+'_clsfWeights_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_'+task_name+'_activations.png'
        
    #plt.savefig(plotname)
    plt.show()
    
    return plt, evoked.data


### Get the time index of the max classifier score from PointByPoint_MVPA.ipynb file!!!!!

In [ ]:
epochs.times[39]

In [ ]:
maxClassificationScore_timeIndex =  39 
clsf_group, act_group = [], []
for i in range(len(s_id_list)):
    print('------- ' + s_id_list[i] + ' -------')
    model = models_list[i][0] # get the element at the index 0 since each model is kept in a sublist e.g. [[m1], [m2]]
    
    #Plot Classifier weights
    plt_1, clsf_tmp = plotActivations(s_id_list[i], model, epochs_list[i], 
                                      timePoint=maxClassificationScore_timeIndex, isClsf=True, standardize=True)
    clsf_group.append(clsf_tmp)
    #plot activation maps
    plt_2, act_tmp = plotActivations(s_id_list[i], model, epochs_list[i],
                                     timePoint=maxClassificationScore_timeIndex, isClsf=False, standardize=True)
    act_group.append(act_tmp)
    
    

### Average classifier weights

In [ ]:
clsf_group_mean = np.mean(clsf_group, axis=0)
print('mean clsf group shape: ', clsf_group_mean.shape)
evoked = mne.EvokedArray(clsf_group_mean, epochs_list[0].info, tmin=0)
print( epochs_list[0].info)
evoked.plot_topomap([0.0], show=False, time_unit='s', outlines="head", size=4, colorbar=True,
                    ch_type="mag", show_names=False)

plotname = figuresFolder + 'all_clsfWeights_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_' + task_name + '_clsfWeights.png'
plt.savefig(plotname)
plt.show()

### Average activation map at 0.29s (ind=39)

In [ ]:
act_group_mean = np.mean(act_group, axis=0)
print('mean act group shape: ',act_group_mean.shape)
evoked = mne.EvokedArray(act_group_mean, epochs_list[0].info, tmin=0)
print( epochs_list[0].info)
evoked.plot_topomap([0.0], show=False, time_unit='s', outlines="head", size=4, colorbar=True,
                    ch_type="mag", show_names=False) #, cmap='seismic')

plotname = figuresFolder + 'all_clsfWeights_'+str(peak_indices[0])+'_'+str(peak_indices[1])+'_' + task_name + '_activations.png'

plt.savefig(plotname)
plt.show()

In [ ]:
max(evoked._data)